In [2]:
from sqlalchemy import types
from StringIO import StringIO
import sqlite3
import CleanData as cd
import numpy as np
import pandas as pd

# Clean Data

### British Museum

In [50]:
title = 'Roman_Imperial_Coinage'

In [57]:
# Read in data and convert to appropriate types
bm_df = pd.read_csv('../Data/BM_'+title+'.csv',
                 converters={"Authority": cd.stringToList(), 'Associated names': cd.stringToList(), 
                             'Subjects': cd.stringToList(), 'Inscriptions': cd.stringToListofDicts('|', ';', ':')})
bm_df = pd.concat([bm_df, cd.cleanInscriptions(bm_df)], axis=1)
bm_df = bm_df.drop('Unnamed: 0', 1)
bm_df = bm_df.replace(np.nan, '', regex=True)
bm_df = bm_df.rename(columns = {'Culture/period':'Culture'})

In [58]:
# Clean data to only retain relevant information
mask = ((bm_df['Object type'] == 'coin ') & (bm_df['Date'].str.find('stC') == -1))
filtered = bm_df[mask]

lists = ['Authority', 'Subjects', 'Associated names', 'Obverse legend', 'Reverse legend']
strings = ['Museum number', 'Denomination', 'Description', 'State', 'Culture', 'Materials', 
            'Curator\'s comments', 'Bibliography', 'Object type', "Production place"]
floats = ['Weight (g)']
dates = ['Date']
redundant_notes = ['Production place', 'Denomination']
do_nothing = ['url', 'Inscriptions']
duplicate_cols = ['Authority', 'Date', 'Production place', 'Description', 'Subjects', "Curator's comments",
                  'Obverse legend', 'Reverse legend']

cleaned_bm = cd.cleanDF(filtered, lists, strings, floats, dates, redundant_notes, do_nothing, duplicate_cols)
cleaned_bm = cleaned_bm.drop('Inscriptions', 1)
cleaned_bm['Source'] = 'British Museum'
#cleaned_bm.head()

In [59]:
# Convert everything to strings and integers to put into SQL db
lists = ['Authority', 'Subjects', 'Associated names', 'Obverse legend', 'Reverse legend', 'Date']
for col in lists:
    cleaned_bm[col] = cleaned_bm[col].apply(cd.listToString())

columns = [u'Associated names', u'Authority', u'Bibliography', u'Culture',
       u'Curator\'s comments', u'Date', u'Denomination', u'Description',
       u'Materials', u'Museum number', u'Object type', u'Obverse legend',
       u'Production place', u'Reverse legend', u'State', u'Subjects',
       u'url', u'Source']

for col in columns:
    cleaned_bm[col] = cleaned_bm[col].apply(lambda x: x.decode('utf-8'))

cleaned_bm.columns = ['associatedNames', 'authority', 'bibliography', 'culture',
                       'curatorComment', 'date', 'denomination', 'description',
                       'materials', 'museumNumber', 'objectType', 'obverseLegend',
                       'mint', 'reverseLegend', 'state', 'subjects',
                       'weight', 'url', 'source']

cleaned_bm['startDate'] = cleaned_bm.apply(lambda row: row['date'].split(",")[0], axis=1)
cleaned_bm['endDate'] = cleaned_bm.apply(lambda row: row['date'].split(",")[1] 
                                                 if len(row['date'].split(",")) > 1 
                                                 else row['date'].split(",")[0], 
                                         axis=1)

In [60]:
cleaned_bm.tail()

,associatedNames,authority,bibliography,culture,curatorComment,date,denomination,description,materials,museumNumber,...,obverseLegend,mint,reverseLegend,state,subjects,weight,url,source,startDate,endDate
22024,"Antinous, Demeter/Ceres",Hadrian,"BMC Greek (Mysia) 13, p.4 PCG 86.3, p. (pl. 47...",Roman Provincial,The young courtier Antinous died suddenly in A...,"130, 138",?,"Alloy coin.(obverse) Bare head of Antinous, ri...",alloy,"1844,0425.1609",...,AHTINOOC IAKXOC,Adramyteum,ECIOC AN HKE A PAMVTHNOIC,Roman Empire,classical deity,26.69,http://www.britishmuseum.org/research/collecti...,British Museum,130,138
22025,Constantine the Great,Constantine the Great,"RIC6 65, p.407 (type)",Roman Imperial,,"312, 313",solidus,"Gold coin.(obverse) Bust of Constantine I, dra...",gold,"1896,0608.98",...,IMP CONSTANTINVS P F AVG,Ostia,PRINCIPI IVVENTVTIS,Roman Empire,emperor/empress,8.49,http://www.britishmuseum.org/research/collecti...,British Museum,312,313
22026,Hadrian,Hadrian,"RE3 1209, p. 417 Strack (Hadrian) 557 (var = f...",Roman Imperial,,"117, 138",sestertius,"on the left, three citizens in front and one b...",copper alloy,"1869,0507.7",...,IMP CAESAR TRAIANVS HADRIANVS AVG P M TR P COS...,Rome,RELIQVA VETERA HS N[OVIES] MILL ABOLITA,Roman Empire,"politics, emperor/empress",24.07,http://www.britishmuseum.org/research/collecti...,British Museum,117,138
22027,Hadrian,Hadrian,"RE3 1207, p. 417 Strack (Hadrian) 557 RIC2 592...",Roman Imperial,,"117, 138",sestertius,"on the left, three citizens (two in front, one...",copper alloy,"1872,0709.561",...,IMP CAESAR TRAIANVS HADRIANVS AVG P M TR P COS...,Rome,RELIQVA VETERA HS NOVIES MILL ABOLITA,Roman Empire,"politics, emperor/empress",20.64,http://www.britishmuseum.org/research/collecti...,British Museum,117,138
22028,Constantine the Great,Constantine the Great,RIC7 p275.381,Roman Imperial,maiorina (big [coins]) and centenionalis (a ‘h...,334,nummus,"Copper alloy coin.(obverse) Diademed, draped a...",copper alloy,B.1746,...,CONSTANTINVS MAX AVG,Arles,GLORIA EXERCITVS,Roman Empire,"soldier, emperor/empress",2.28,http://www.britishmuseum.org/research/collecti...,British Museum,334,334


### American Numismatic Society

In [9]:
ans_df = pd.read_csv('../Data/ANS_'+title+'.csv',
                    converters={"Authority": cd.stringToList('|'), 'Year': cd.stringToList('|'),
                               'Issuer': cd.stringToList('|'), 'Portrait': cd.stringToList('|'),
                               'Reference': cd.stringToList('|')})
ans_df = ans_df.replace(np.nan, '', regex=True)
ans_df['Description'] = ans_df.apply(lambda row: cd.makeDescription(row['Material'], row['Manufacture'], row['Obverse Type'], row['Reverse Type']), axis=1)
ans_df.tail()

,Unnamed: 0,URI,Title,RecordId,Authority,Coin Type URI,Date on Object,Degree,Deity,Denomination,...,Reference,Region,Reverse Legend,Reverse Type,Weight,Year,Thumbnail_obv,Thumbnail_rev,Date Record Modified,Description
62995,62995,http://numismatics.org/collection/1944.100.21593,"Bronze AE3 of Constantius II, Cyzicus, AD 337 ...",1944.100.21593,[Constantius II],http://numismatics.org/ocre/id/ric.8.cyz.18,,,,AE3,...,[RIC VIII Cyzicus 18],Mysia,GLOR-IA EXERC-ITVS,Two soldiers standing holding shield and spear...,1.6,"[337, 340]",http://numismatics.org/collectionimages/190019...,http://numismatics.org/collectionimages/190019...,2017-09-08T14:22:22Z,Struck Billon|Bronze. (obverse) Head laureate ...
62996,62996,http://numismatics.org/collection/1944.100.21594,"Bronze AE3 of Constantius II, Cyzicus, AD 337 ...",1944.100.21594,[Constantius II],http://numismatics.org/ocre/id/ric.8.cyz.18,,,,AE3,...,[RIC VIII Cyzicus 18],Mysia,GLOR-IA EXERC-ITVS,Two soldiers standing holding shield and spear...,1.36,"[337, 340]",http://numismatics.org/collectionimages/190019...,http://numismatics.org/collectionimages/190019...,2017-09-08T14:22:22Z,Struck Billon|Bronze. (obverse) Head laureate ...
62997,62997,http://numismatics.org/collection/1944.100.21595,"Bronze AE3 of Constantius II, Cyzicus, AD 337 ...",1944.100.21595,[Constantius II],http://numismatics.org/ocre/id/ric.8.cyz.18,,,,AE3,...,[RIC VIII Cyzicus 18],Mysia,GLOR-IA EXERC-ITVS,Two soldiers standing holding shield and spear...,1.55,"[337, 340]",http://numismatics.org/collectionimages/190019...,http://numismatics.org/collectionimages/190019...,2017-09-08T14:22:22Z,Struck Billon|Bronze. (obverse) Head laureate ...
62998,62998,http://numismatics.org/collection/1944.100.21596,"Bronze AE3 of Constantius II, Cyzicus, AD 337 ...",1944.100.21596,[Constantius II],http://numismatics.org/ocre/id/ric.8.cyz.18,,,,AE3,...,[RIC VIII Cyzicus 18],Mysia,GLOR-IA EXERC-ITVS,Two soldiers standing holding shield and spear...,1.44,"[337, 340]",http://numismatics.org/collectionimages/190019...,http://numismatics.org/collectionimages/190019...,2017-09-08T14:22:22Z,Struck Billon|Bronze. (obverse) Head laureate ...
62999,62999,http://numismatics.org/collection/1944.100.21597,"Bronze AE3 of Constantius II, Cyzicus, AD 337 ...",1944.100.21597,[Constantius II],http://numismatics.org/ocre/id/ric.8.cyz.18,,,,AE3,...,[RIC VIII Cyzicus 18],Mysia,GLOR-IA EXE[RC]-ITVS,Two soldiers standing holding shield and spear...,1.96,"[337, 340]",http://numismatics.org/collectionimages/190019...,http://numismatics.org/collectionimages/190019...,2017-09-08T14:22:22Z,Struck Billon|Bronze. (obverse) Head laureate ...


In [10]:
lists = ['Year', 'Authority', 'Issuer', 'Portrait', 'Reference']
strings = ['Deity', 'Denomination', 'Mint', 'Description', 'Obverse Legend', 'Reverse Legend']
floats = ['Weight']
dates = []
redundant_notes = []
do_nothing = ['URI']
duplicate_cols = ['Issuer', 'Year', 'Mint', 'Deity', 'Description', 'Obverse Legend', 'Reverse Legend']

cleaned_ans = cd.cleanDF(ans_df, lists, strings, floats, dates, redundant_notes, do_nothing, 
                              duplicate_cols, production_place='Mint')
cleaned_ans['Source'] = 'American Numismatic Society'
cleaned_ans.tail()
cleaned_ans.columns

Index([u'Authority', u'Deity', u'Denomination', u'Description', u'Issuer',
       u'Mint', u'Obverse Legend', u'Portrait', u'Reference',
       u'Reverse Legend', u'URI', u'Weight', u'Year', u'Source'],
      dtype='object')

In [11]:
# Convert everything to strings and integers to put into SQL db
lists = ['Year', 'Authority', 'Issuer', 'Portrait', 'Reference']
for col in lists:
    cleaned_ans[col] = cleaned_ans[col].apply(cd.listToString())
    
cleaned_ans['StartDate'] = cleaned_ans.apply(lambda row: row['Year'].split(",")[0], axis=1)
cleaned_ans['EndDate'] = cleaned_ans.apply(lambda row: row['Year'].split(",")[1] 
                                                 if len(row['Year'].split(",")) > 1 
                                                 else row['Year'].split(",")[0], 
                                         axis=1)

columns = [u'Authority', u'Deity', u'Denomination', u'Description', 'EndDate', u'Issuer',
       u'Mint', u'Obverse Legend', u'Portrait', u'Reference',
       u'Reverse Legend', u'URI', u'Year', 'StartDate', u'Source']

cleaned_ans.columns = columns

for col in columns:
    cleaned_ans[col] = cleaned_ans[col].apply(lambda x: x.decode('utf-8'))

In [13]:
cleaned_ans.tail()

,Authority,Deity,Denomination,Description,Issuer,Mint,Obverse Legend,Portrait,Reference,Reverse Legend,URI,Weight,Year,Source,startDate,endDate
721,Constantius II,,AE3,Struck Billon|Bronze. (obverse) Head laureate ...,,Cyzicus,D N CONSTA-NS P F AVG,Constans,RIC VIII Cyzicus 18,[G]LOR-IA EXERC-ITVS,http://numismatics.org/collection/1944.100.21585,1.65,"337, 340",British Museum,337,340
722,Constantius II,,AE3,Struck Billon|Bronze. (obverse) Head laureate ...,,Cyzicus,D N CONS[TA]-NS P F AVG,Constans,RIC VIII Cyzicus 18,[G]LOR-IA EXERC-ITVS,http://numismatics.org/collection/1944.100.21587,1.21,"337, 340",British Museum,337,340
723,Constantius II,,AE3,Struck Billon|Bronze. (obverse) Head laureate ...,,Cyzicus,D N CO[NSTA]-NS P F AVG,Constans,RIC VIII Cyzicus 18,GLOR-[IA] EXERC-ITVS,http://numismatics.org/collection/1944.100.21589,1.81,"337, 340",British Museum,337,340
724,Constantius II,,AE3,Struck Billon|Bronze. (obverse) Head laureate ...,,Cyzicus,D N CONST[A]-NS P F AVG,Constans,RIC VIII Cyzicus 18,GLOR-IA EXERC-ITVS,http://numismatics.org/collection/1944.100.21596,1.44,"337, 340",British Museum,337,340
725,Constantius II,,AE3,Struck Billon|Bronze. (obverse) Head laureate ...,,Cyzicus,D N CONST[A-N]S P F AVG,Constans,RIC VIII Cyzicus 18,GLOR-IA EXE[RC]-ITVS,http://numismatics.org/collection/1944.100.21597,1.96,"337, 340",British Museum,337,340


# Store Data

In [74]:
cnx = sqlite3.connect('../Data/'+title+'.sqlite')
cur = cnx.cursor()

In [75]:
cur.execute('''CREATE TABLE britishMuseum(associatedNames TEXT, authority TEXT, bibliography TEXT, culture TEXT,
                    curatorComment TEXT, dates TEXT, denomination TEXT, 
                    description TEXT, materials TEXT, museumNumber TEXT, objectType TEXT, obverseLegend TEXT,
                    mint TEXT, reverseLegend TEXT, state TEXT, subjects TEXT, weight INT,
                    url TEXT, source TEXT, startDate INT, endDate INT)''')

In [68]:
cleaned_bm.loc[1]

associatedNames                                                     
authority                                                 Magnentius
bibliography                                                        
culture                                               Roman Imperial
curatorComment                                                      
date                                                        350, 353
denomination                                                       ?
description                                             Silver coin.
materials                                                     silver
museumNumber                                             1981,0745.6
objectType                                                      coin
obverseLegend                                                       
mint                                                        Aquileia
reverseLegend                                                       
state                             

In [69]:
list(cleaned_bm.to_records(index=False))[1]

(u'', u'Magnentius', u'', u'Roman Imperial', u'', u'350, 353', u'?', u'Silver coin.', u'silver', u'1981,0745.6', u'coin', u'', u'Aquileia', u'', u'Roman Empire', u'', 2.78, u'http://www.britishmuseum.org/research/collection_online/collection_object_details.aspx?objectId=2880&partId=1&searchText=roman+imperial+coins&lookup-people=e.g.+Hokusai%2c+Ramesses&people=&lookup-place=e.g.+India%2c+Shanghai%2c+Thebes&place=&from=ad&fromDate=&to=ad&toDate=&lookup-object=e.g.+bowl%2c+hanging+scroll%2c+print&object=&lookup-subject=e.g.+farming%2c+New+Testament&subject=&lookup-matcult=e.g.+Choson+Dynasty%2c+Ptolemaic&matcult=&lookup-technique=e.g.+carved%2c+celadon-glazed&technique=&lookup-school=e.g.+French%2c+Mughal+Style&school=&lookup-material=e.g.+canvas%2c+porcelain%2c+silk&material=&lookup-ethname=e.g.+Hmong%2c+Maori%2c+Tai&ethname=&lookup-ware=e.g.+Imari+ware%2c+Qingbai+ware&ware=&lookup-escape=e.g.+cylinder%2c+gravity%2c+lever&escape=&lookup-bibliography=&bibliography=&citation=&museumno=&ca

In [76]:
cur.executemany("""INSERT INTO britishMuseum (associatedNames, authority, bibliography, culture,
                        curatorComment, dates, denomination, description,
                        materials, museumNumber, objectType, obverseLegend,
                        mint, reverseLegend, state, subjects, weight,
                        url, source, startDate, endDate) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", 
                list(cleaned_bm.to_records(index=False)))

In [78]:
cur.execute('''CREATE TABLE americanNumismaticSociety(authority TEXT, deity TEXT, denomination TEXT, description TEXT, 
                       issuer TEXT, mint TEXT, obverseLegend TEXT, portrait TEXT, reference TEXT, reverseLegend TEXT, 
                       url TEXT, weight INT, dates TEXT, source TEXT, startDate INT, endDate INT)''')

In [79]:
cur.executemany("""INSERT INTO americanNumismaticSociety (authority, deity, denomination, description, 
                       issuer, mint, obverseLegend, portrait, reference, reverseLegend, url, weight, dates, 
                       source, startDate, endDate) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", 
                list(cleaned_ans.to_records(index=False)))

In [80]:
cnx.commit()